In [ ]:
# shif audio
def time_shift(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(np.random.random() * shift_limit * sig_len)
    return (np.roll(sig,shift_amt), sr)

In [ ]:
def pad_trunc(aud):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = 124665

    if (sig_len > 124665):
      # Truncate the signal to the given length
      sig = sig[:,:max_len]

    elif (sig_len < 124665):
      # Length of padding to add at the beginning and end of the signal
        pad_begin_len = np.random.randint(0, max_len - sig_len)
        pad_end_len = max_len - sig_len - pad_begin_len

      # Pad with 0s
        pad_begin = torch.zeros((num_rows, pad_begin_len))
        sig=torch.tensor(sig)
        pad_end = torch.zeros((num_rows, pad_end_len))
        sig = torch.cat((pad_begin, sig, pad_end), 1)
      
    return (sig, sr)

In [ ]:
import torch
import librosa
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# play audio

In [ ]:
# Play an audio file

testpath='../input/moroccan-darija-trigger-word-classification-ed-2/data/test/'
trainpath = '../input/moroccan-darija-trigger-word-classification-ed-2/data/train/'

        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def load_data(a):
    spec=[1,2]
    y,sr=pad_trunc(librosa.load(a+".wav",sr=48000,mono=False))
    y2,sr2=time_shift([y,sr],1)
    spec[0],spec[1]=librosa.feature.melspectrogram(y2[0],sr=48000),librosa.feature.melspectrogram(y2[1],sr=48000)
    spec[0],spec[1]=librosa.amplitude_to_db(spec[0]),librosa.amplitude_to_db(spec[1])
    spec=np.array(spec)
    return spec

# read data

In [ ]:
data=pd.read_csv('../input/moroccan-darija-trigger-word-classification-ed-2/train.csv')

# split data

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(data['id'], data['label'], test_size=0.2, random_state=42)

# prepare data

In [ ]:
for i in range(0,X_train.shape[0]):
    X_train.iloc[i]=load_data(str(trainpath)+str(X_train.iloc[i]))
    
    

In [ ]:
for i in range(0,X_test.shape[0]):
    X_test.iloc[i]=load_data(str(trainpath)+str(X_test.iloc[i]))

In [ ]:
a=np.array([*X_train.values])
train =a.reshape(-1,128,244,2)
b=np.array([*X_test.values])
test=b.reshape(-1,128,244,2)

# model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
y_binary =keras.utils.to_categorical(y_train)
y_t=keras.utils.to_categorical(y_test)

In [ ]:
model=keras.Sequential()
model.add(layers.Conv2D(24,(5,5),activation='relu',input_shape=train.shape[1:]))
model.add(layers.MaxPooling2D((4,2)))
model.add(layers.Conv2D(48,(5,5),activation='relu'))
model.add(layers.MaxPooling2D((4,2)))
model.add(layers.Conv2D(48,(5,5),activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2,activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train,y_binary,epochs=20,batch_size=10)

In [ ]:
results = model.predict(test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(results>0.5,y_t)

# subbmision

In [ ]:
test=pd.read_csv('../input/moroccan-darija-trigger-word-classification-ed-2/test.csv')

In [ ]:
for i in range(0,test['id'].shape[0]):
    test['id'].iloc[i]=load_data(str(testpath)+str(test['id'].iloc[i]))

In [ ]:
g=np.array([*test['id'].values])
pre =g.reshape(-1,128,244,2)

In [ ]:
results=model.predict(pre)

In [ ]:
pred=np.argmax(results,axis=1)

In [ ]:
sub=pd.read_csv('../input/moroccan-darija-trigger-word-classification-ed-2/sample_submission.csv')

In [ ]:
sub['label']=pred

In [ ]:
sub.to_csv('sub.csv',index=False)